In [4]:
!pip install opencv-python
!pip install pillow

     ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
     --------- ------------------------------ 0.6/2.5 MB 12.6 MB/s eta 0:00:01
     ------------------ --------------------- 1.2/2.5 MB 12.5 MB/s eta 0:00:01
     ------------------------------ --------- 2.0/2.5 MB 13.9 MB/s eta 0:00:01
     ---------------------------------------  2.5/2.5 MB 14.6 MB/s eta 0:00:01
     ---------------------------------------  2.5/2.5 MB 14.6 MB/s eta 0:00:01
     ---------------------------------------  2.5/2.5 MB 14.6 MB/s eta 0:00:01
     ---------------------------------------  2.5/2.5 MB 14.6 MB/s eta 0:00:01
     ---------------------------------------- 2.5/2.5 MB 7.0 MB/s eta 0:00:00


In [16]:
from PIL import Image
import os
import random

# Set the path to your object images
objects_path = "individualItem"

# Load the object images
object_files = os.listdir(objects_path)
objects = [Image.open(os.path.join(objects_path, f)).convert("RGBA") for f in object_files]

# Set the path to your background image
background_path = "00-EmptyFridge.png"

# Load the background image
background = Image.open(background_path).convert("RGBA")

# Set the maximum number of objects to place on the background
max_objects = 10

# Loop through each object image and randomly place it on the background
placed_objects = []
def placeFridgeItems(path, a):
    for i in range(max_objects):
        # Choose a random object image that hasn't already been placed
        available_objects = [o for o in objects if o not in placed_objects]
        if not available_objects:
            break  # if all objects have been placed, stop placing more objects
        object_img = random.choice(available_objects)

        # Get the dimensions of the object image
        object_width, object_height = object_img.size

        # Choose a random location on the background to place the object
        while True:
            x = random.randint(0, background.width - object_width)
            y = random.randint(0, background.height - object_height)

            # Check if the object overlaps with any previously placed object
            object_box = (x, y, x+object_width, y+object_height)
            overlap = False
            for placed_obj_box in map(lambda obj: obj.getbbox(), placed_objects):
                if placed_obj_box and Image.new("RGBA", object_img.size).crop(placed_obj_box).getbbox():
                    if object_box[0] < placed_obj_box[2] and object_box[2] > placed_obj_box[0] \
                            and object_box[1] < placed_obj_box[3] and object_box[3] > placed_obj_box[1]:
                        overlap = True
                        break
            if not overlap:
                break

        # Paste the object on the background
        background.alpha_composite(object_img, (x, y))

        # Add the placed object to the list of placed objects
        placed_objects.append(object_img)

    # Save the modified image in sample fridge, make sure to change the name if it already exists
    background.save(path + "samplefridge" + str(a) + ".png")


path = "sampleFridge/"
#check if the name exists in the path
for a in range(100):
    placeFridgeItems(path, a)
